# Home Credit Default Risk by Kaggle

Inspired by the classic Kaggle problem: [Home Credit Default Risk](https://www.kaggle.com/competitions/home-credit-default-risk/overview), we are creating an ultimate cheatsheet for data scientist interviews. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, cross_val_predict

from sklearn import set_config
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from xgboost import XGBClassifier, plot_importance
from scipy.stats import loguniform, randint
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve, classification_report
from outlier_and_nan_imputer import OutlierAndNaNImputer



set_config(transform_output="pandas") 

In [2]:
raw_data = pd.read_csv("data/application_train.csv")

df = raw_data.copy()
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


# 1. Check the target column

In [3]:
# 1. Check target value
df.groupby("TARGET").size()

TARGET
0    282686
1     24825
dtype: int64

In [4]:
df.drop(columns = ["TARGET", "SK_ID_CURR"], inplace=True)
trans_values = {}
col_transformers = []

This is a very **imbalanced data**, with label 1 less than 10% of label 0.
The way we usually handle imbalance data:

a) Data level:
Collect more data in minority class  
Undersample: might remove important data  
Oversample: might overfit  
To mitigate:   
Two-phase learning: train on resampled data, fine tune on original data  
Dynamic sampling: oversample the low-performing class and undersample the high performing class during training.  

b) Metric level: Check precision/recall, f1, AUC in ROC curve

c) Algorithm level: Add stronger penalty on misclassifying the minority class in loss function

# 2. Check predictors


2.0 Check the dtypes of predictors  
2.1 Check if all the numeric binary columns are in [0, 1]  
2.2 Check outliers in non-binary numeric cols, and impute them with null, medium or mean

In [5]:
# 2. Check the predictors
df.dtypes.value_counts()

float64    65
int64      39
object     16
Name: count, dtype: int64

In [6]:
num_cols = df.select_dtypes(include='number').columns.tolist()
num_cols = [_ for _ in num_cols if _ not in ["SK_ID_CURR", "TARGET"]]
print(len(num_cols))

104


In [7]:
num_cols_nonbinary = [x for x in num_cols if all(words not in x for words in ["FLAG", "NOT"])]
print(len(num_cols_nonbinary))
num_cols_binary = [_ for _ in num_cols if _ not in num_cols_nonbinary]
print(len(num_cols_binary))

72
32


In [8]:
# 2.1 Check if all the numeric binary columns are either 0 or 1
invalid_cols = df[num_cols_binary].columns[~df[num_cols_binary].isin([0,1]).all()]
print(invalid_cols)
# Looks good!

Index([], dtype='object')


In [9]:
cat_cols = df.select_dtypes(include='object').columns.tolist()

Unlike Logistic Regression and linear regression models, xgboost handles missing values and outliers natively.

**Note!** In real-life ML practice, we always use the **training data only** for imputation or outlier detection. We can not foresee the stats of future unseen data in practice, and using test data would cause data leakage.  
In a Kaggle competition, however, we use train+test stats as it generally improves leaderboard score, but it won't generalize well outside the leaderboard.  
This notebook is for data scientist interview prep, so we always stick to real-life practice. The stats from training data are stored in a dictionary and will be applied to test data later.




# 3. Missing values in predictors

**Tree-based models**, like XGBoost, can handle missing values and no need to inpute.  
**Distance-based models**, linear Regression, SVM, KNN, Neural networks need imputation.



# 4. Feature Engineering

## 4.1 One-hot vector

In real pipeline, we use SimpleImputer to avoid dataleakage.

# Modeling

In [10]:
# Prepare features

Y = raw_data["TARGET"]
X_cols = [_ for _ in df.columns if _ not in ["TARGET", "SK_ID_CURR"]]
X = df[X_cols]


In [11]:
# Other column transformers


other_cat_cols_pipline = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
]
)


col_transformers = [
    ("other_cat", other_cat_cols_pipline, cat_cols)
    
]

preprocessor = ColumnTransformer(transformers= col_transformers, remainder="passthrough" )#.set_output(transform="pandas") 


In [12]:
X_cleaned = preprocessor.fit_transform(X)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X_cleaned, Y, test_size=0.2, random_state=42, stratify=Y
)

# === 2) Class imbalance handling (on TRAIN only)
pos = int(np.sum(y_train == 1))
neg = int(np.sum(y_train == 0))
spw = neg / max(pos, 1)
print("scale_pos_weight:", round(spw, 3))

scale_pos_weight: 11.387


In [15]:
# === 3) Hyperparameter search (AUC)
base = XGBClassifier(
    objective="binary:logistic",
    max_depth=6,
    eval_metric="auc",
    # tree_method="hist",
    random_state=42,
    learning_rate=0.1,      # fixed
    subsample=0.8,          # fixed
    colsample_bytree=0.8,   # fixed
    # min_child_weight=1,     # fixed
    # gamma=0.0,              # fixed
    # reg_alpha=0.0,          # fixed
    # reg_lambda=1.0,         # fixed
    use_label_encoder=False,
    scale_pos_weight=spw
)


param_dist = {
    "n_estimators": [300, 1000],
}


cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
search = RandomizedSearchCV(
    estimator=base,
    param_distributions=param_dist,
    n_iter=2,
    scoring="roc_auc",
    n_jobs=-1,
    cv=cv,
    verbose=1,
    refit=True     # refit on full TRAIN with best params (still fine for Option A)
)
search.fit(X_train, y_train)

print("Best AUC (CV):", search.best_score_)
print("Best params:", search.best_params_)



Fitting 3 folds for each of 2 candidates, totalling 6 fits


C:\Users\xinxin\venvs\ml311\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:28:53] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best AUC (CV): 0.7454352130212233
Best params: {'n_estimators': 300}


In [16]:
# === 4) Build a clean estimator with best params for OOF predictions
best = search.best_estimator_

# 5) OOF probs for threshold selection (no leakage)
oof_probs = cross_val_predict(
    best, X_train, y_train, cv=cv, method="predict_proba", n_jobs=-1
)[:, 1]

prec, rec, thr = precision_recall_curve(y_train, oof_probs)
f1s = 2 * prec * rec / (prec + rec + 1e-12)
best_idx = int(np.nanargmax(f1s))
best_thresh = float(thr[max(best_idx - 1, 0)])
print("Chosen threshold (OOF):", round(best_thresh, 4))
print("OOF F1:", round(float(f1s[best_idx]), 4))

# 6) Fit on all TRAIN and evaluate on TEST
best.fit(X_train, y_train)
test_probs = best.predict_proba(X_test)[:, 1]
test_auc = roc_auc_score(y_test, test_probs)
test_pred = (test_probs >= best_thresh).astype(int)

print("\n=== TEST METRICS (threshold tuned on OOF) ===")
print("ROC AUC:", round(float(test_auc), 4))
print("F1:", round(float(f1_score(y_test, test_pred)), 4))
print("Confusion matrix:\n", confusion_matrix(y_test, test_pred))
print("\nClassification report:\n", classification_report(y_test, test_pred, digits=4))

Chosen threshold (OOF): 0.612
OOF F1: 0.2942


C:\Users\xinxin\venvs\ml311\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:30:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== TEST METRICS (threshold tuned on OOF) ===
ROC AUC: 0.7578
F1: 0.3058
Confusion matrix:
 [[48785  7753]
 [ 2669  2296]]

Classification report:
               precision    recall  f1-score   support

           0     0.9481    0.8629    0.9035     56538
           1     0.2285    0.4624    0.3058      4965

    accuracy                         0.8305     61503
   macro avg     0.5883    0.6627    0.6047     61503
weighted avg     0.8900    0.8305    0.8552     61503



In [17]:
plot_importance(best, max_num_features=15)
plt.show()

The roc-auc score is fine but f1-score is low. One possible reason is the default threshold 0.5 didn't cutoff properly.

# Submit

In [18]:
raw_test_data = pd.read_csv("data/application_test.csv")


In [20]:
test_df = raw_test_data[[_ for _ in raw_test_data.columns if _!= "SK_ID_CURR"]]
test_df = preprocessor.transform(test_df)

In [22]:
test_y_pred = best.predict(test_df)
test_y_pred_prob = best.predict_proba(test_df)[:, 1]


final = pd.DataFrame(raw_test_data[["SK_ID_CURR"]])
final["TARGET"] = test_y_pred_prob

final.to_csv("submission_xgboost.csv", index=False, header=True)

This version got a 0.74 AUC score too...